In [17]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform
import re 
import nibabel as nib
import statistics
import sys

import mri_data
import monai_training

from reload_recursive import reload_recursive

In [18]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, Scan  # noqa: E402, F401
from mri_data import utils
from mri_data import file_manager as fm
from monai_training import preprocess  # noqa: E402
from monai_training.preprocess import DataSetProcesser

In [3]:
logger.remove()
logger.add(sys.stderr, level="INFO")

1

In [8]:
#! Set these variables
work_dir_name = "pituitary1"
label_prefix = "pituitary"

I think I have function for part of the below

In [12]:
hostname = platform.node()
if hostname == "rhinocampus":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
ensemble_out_dir = work_dir / "ensemble_output" / "3Tpioneer_bids"

Scan the ensemble_out folder produced during the monai training to collect the predicted labels into a DataSet struct. Create another DataSet object with the same subjects and sessions but rooted in the main data location, then collect the manual labels 

In [14]:
ensemble_out_dataset = scan_3Tpioneer_bids(ensemble_out_dir, label="t1_ensemble.nii.gz")
dataset = DataSet.dataset_like(ensemble_out_dataset)
dataset.dataroot = dataroot
dataset_proc = DataSetProcesser(dataset)
dataset_proc.prepare_labels("pituitary")
dataset = dataset_proc.dataset
dataset.sort()
ensemble_out_dataset.sort()

2024-12-02 14:56:54.168 | INFO     | monai_training.preprocess:prepare_labels:167 - Prepare Labels
  0%|          | 0/13 [00:00<?, ?it/s]

100%|██████████| 13/13 [00:01<00:00,  9.60it/s]
2024-12-02 14:56:55.525 | INFO     | monai_training.preprocess:prepare_labels:229 - Dataset size: 13


Calculate Dice Scores

In [22]:
dice_scores = []
print("Dice Scores")
for scan, inference in zip(dataset, ensemble_out_dataset):
    seg1 = nib.load(scan.label_path).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.dice_score(seg1, seg2)
    dice_scores.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(dice_scores) / len(dice_scores)))
print("Std: {:0.2f}".format(statistics.stdev(dice_scores)))

print(dice_scores)

Dice Scores
1029: 0.83
1037: 0.85
1052: 0.78
1057: 0.79
1103: 0.77
1111: 0.90
1207: 0.82
1224: 0.80
1241: 0.77
1248: 0.80
1323: 0.89
1401: 0.78
1492: 0.85
----------
Mean: 0.82
Std: 0.04
[0.8340141106572596, 0.8498603351955307, 0.776134625191229, 0.7908337396392003, 0.7660343270099368, 0.9001782531194296, 0.8176100628930818, 0.803123983078425, 0.7672842295526434, 0.8004330566582462, 0.8885424785658612, 0.7830775611779345, 0.8453934361407671]


Calculate IoU

In [23]:
iou = []
print("IoU")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.iou(seg1, seg2, seg2_val=1)
    iou.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(iou) / len(iou)))
print("Std: {:0.2f}".format(statistics.stdev(iou)))

print(iou)

IoU
1029: 0.72
1037: 0.74
1052: 0.63
1057: 0.65
1103: 0.62
1111: 0.82
1207: 0.69
1224: 0.67
1241: 0.62
1248: 0.67
1323: 0.80
1401: 0.64
1492: 0.73
----------
Mean: 0.69
Std: 0.06
[0.7152866242038216, 0.7389192471159685, 0.6341666666666667, 0.6540322580645161, 0.6207906295754027, 0.8184764991896273, 0.6914893617021277, 0.6710168569874932, 0.6224340175953079, 0.6672683513838749, 0.7994389901823282, 0.643490115882754, 0.7321917808219178]


Calculate Hausdorff Dist for pituitary

In [24]:
haus = []
print("haus")
for scan, inference in zip(dataset, ensemble_out_dataset):
    label = fm.find_label(scan, "pituitary", ["CH", "ED", "DT"])
    seg1 = nib.load(label).get_fdata()
    seg2 = nib.load(inference.label_path).get_fdata()

    # Calculate Dice score
    score = utils.hausdorff_dist(seg1, seg2, seg2_val=1)
    haus.append(score)
    print("{}: {:0.2f}".format(scan.subid, score))
print("----------")
print("Mean: {:0.2f}".format(sum(haus) / len(haus)))
print("Std: {:0.2f}".format(statistics.stdev(haus)))

print(haus)

haus
1029: 3.16
1037: 3.16
1052: 3.00
1057: 4.12
1103: 72.21
1111: 18.38
1207: 2.00
1224: 5.74
1241: 5.74
1248: 4.58
1323: 2.45
1401: 2.83
1492: 2.24
----------
Mean: 9.97
Std: 19.18
[3.1622776601683795, 3.1622776601683795, 3.0, 4.123105625617661, 72.20803279414278, 18.384776310850235, 2.0, 5.744562646538029, 5.744562646538029, 4.58257569495584, 2.449489742783178, 2.8284271247461903, 2.23606797749979]


Save a new version of the predicted label that has a labelling index of 2 so that it appears as a different color in itksnap

In [26]:
for inference in ensemble_out_dataset:
    label_path = inference.label_path
    new_label_path = label_path.with_name(f"{fm.nifti_name(label_path)}_val2.nii.gz")
    utils.set_label_value(label_path, new_label_path, 2)

Produce commands to open images and labels in itksnap

In [27]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    flair = scan.with_root()
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.root / new_label_path.name
    print("itksnap", "-g", flair, "-o", t1, "-s", prediction, manual)

itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1029/ses-20170816/t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pituitary.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1037/ses-20210508/t1_ensemble_val2.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1037/ses-20210508/pituitary.nii.gz
itksnap -g /media/smbshare/3Tpioneer_bids/sub-ms1052/ses-20180803/flair.nii.gz -o /media/smbshare/3Tpioneer_bids/sub-ms1052/ses-20180803/t1.nii.gz -s /home/srs-9/Projects/ms_mri/training_work_dirs/pituitary1/ensemble_output/3Tpioneer_bids/sub-ms1052/ses-20180803/t1_ensemble_val2.ni

Produce commands to open images and labels in freeview

In [23]:
for scan, inference in zip(dataset, ensemble_out_dataset):
    flair = scan.root / "flair.nii.gz"
    t1 = scan.root / "t1.nii.gz"
    manual = scan.label_path
    prediction = inference.label_path
    print("freeview", flair, t1, f"{prediction}:colormap=heat", f"{manual}:colormap=heat")

freeview /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1010/ses-20180208/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1010/ses-20180208/pineal-CH.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1029/ses-20170816/flair.t1_ensemble.nii.gz:colormap=heat /media/smbshare/3Tpioneer_bids/sub-ms1029/ses-20170816/pineal-SRS.nii.gz:colormap=heat
freeview /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/flair.nii.gz /media/smbshare/3Tpioneer_bids/sub-ms1188/ses-20200720/t1.nii.gz /home/srs-9/Projects/ms_mri/training_work_dirs/pineal1/ensemble_output/sub-ms1188/ses-20200720/flair.t1_ensemble.nii.gz:colormap=heat